In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install jcopml
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import collections
import warnings

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler,Normalizer,MinMaxScaler,RobustScaler,PolynomialFeatures,PowerTransformer,OneHotEncoder,OrdinalEncoder
from sklearn.model_selection import train_test_split,GridSearchCV,RandomizedSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix,classification_report
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import scipy.stats as st

from jcopml.plot import plot_correlation_matrix,plot_classification_report,plot_confusion_matrix,plot_residual,plot_association_matrix
from jcopml.tuning import grid_search_params as gsp, random_search_params as rsp, bayes_search_params as bsp
from jcopml.feature_importance import mean_score_decrease
from skopt import BayesSearchCV
from jcopml.tuning.space import Integer,Real

warnings.filterwarnings("ignore")

# Import Data

In [ ]:
df = pd.read_csv("/kaggle/input/heart-attack-analysis-prediction-dataset/heart.csv")
df.head()

# EDA

### Check Missing Values

In [ ]:
df.isna().sum()

### Check info and Shape Dataframe

In [ ]:
print(f'Shape Dataframe {df.shape}')
print('-------------------------------')
print(f'Info Dataframe {df.info()}')

### Remove Duplicated values

In [ ]:
duplicate_df = df[df.duplicated()]
print(" number of duplicate rows: ", duplicate_df.shape )

df = df.drop_duplicates()
df.shape

# visualization

In [ ]:
plt.figure(figsize=(10,8))
df['output'].value_counts().plot.bar()
plt.title("Couts Output");

In [ ]:
plt.figure(figsize=(15,10))
plt.subplot(331)
sns.countplot('fbs',data=df,hue='output')
plt.subplot(332)
sns.countplot('sex',data=df,hue='output')
plt.subplot(333)
sns.countplot('cp',data=df,hue='output')
plt.subplot(334)
sns.countplot('restecg',data=df,hue='output');
plt.subplot(335)
sns.countplot('exng',data=df,hue='output');
plt.subplot(336)
sns.countplot('slp',data=df,hue='output');
plt.subplot(337)
sns.countplot('caa',data=df,hue='output');
plt.subplot(338)
sns.countplot('thall',data=df,hue='output');

In [ ]:
fig = plt.figure(figsize=(10,8))
sns.violinplot(x='sex', y='age', hue='output', data=df, split=True, palette={0:'r',1:'g'})
plt.title("SEX & AGE HEART ATTACK");

# Correlation Data

In [ ]:
pearson_corr = df.corr()

figure = plt.figure(figsize=(15,10))
sns.heatmap(pearson_corr,annot=True, vmin=-1, vmax=+1)
plt.title("PEARSON CORRELATION")
plt.show()

# Splitting Data

In [ ]:
X = df.drop(columns='output')
y = df['output']

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,stratify=y,random_state=42)
X_train.shape,X_test.shape,y_train.shape,y_test.shape

# Pipeline

In [ ]:
num_pipe = Pipeline([("scaler",StandardScaler()),
                    ("transform",PowerTransformer(method='yeo-johnson'))])


cat_pipe = Pipeline([("encoder",OneHotEncoder())])

# make a model

## SVC

In [ ]:
preprosesor = ColumnTransformer([("numeric",num_pipe,['age','trtbps','chol','thalachh','oldpeak','thall']),
                              
                               ("category",cat_pipe,['sex','fbs','cp','restecg','exng', 'slp','caa'])])


pipeline = Pipeline([("prep",preprosesor),
                    ("algo",SVC(max_iter=200))])

parameter_svc = {"algo__gamma":np.logspace(-3,3,7),
            "algo__C":np.logspace(-3,3,7)}

svc = GridSearchCV(pipeline,parameter_svc,cv=3,n_jobs=-1,verbose=1)
svc.fit(X_train,y_train)

# Evaluation
print(svc.best_params_)
print(svc.score(X_train,y_train),svc.best_score_,svc.score(X_test,y_test))

## KNN

In [ ]:
preprosesor = ColumnTransformer([("numeric",num_pipe,['age','trtbps','chol','thalachh','oldpeak','thall']),
                              
                               ("category",cat_pipe,['sex','fbs','cp','restecg','exng', 'slp','caa'])])


pipeline = Pipeline([("prep",preprosesor),
                    ("algo",KNeighborsClassifier())])

parameter_knn = {'algo__n_neighbors': np.arange(1,30,2),
 'algo__weights': ['uniform', 'distance'],
 'algo__p': [1, 1.5, 2]}

knn = GridSearchCV(pipeline,parameter_knn,cv=3,n_jobs=-1,verbose=1)
knn.fit(X_train,y_train)

# Evaluation
print(knn.best_params_)
print(knn.score(X_train,y_train),knn.best_score_,knn.score(X_test,y_test))

## Random Forest

In [ ]:
preprosesor = ColumnTransformer([("numeric",num_pipe,['age','trtbps','chol','thalachh','oldpeak','thall']),
                              
                               ("category",cat_pipe,['sex','fbs','cp','restecg','exng', 'slp','caa'])])


pipeline = Pipeline([("prep",preprosesor),
                    ("algo",RandomForestClassifier(random_state=42,n_jobs=-1))])

parameter_rf = {'algo__n_estimators': [100, 150, 200],
 'algo__max_depth': [20, 50, 80],
 'algo__max_features': [0.3, 0.6, 0.8],
 'algo__min_samples_leaf': [1, 5, 10]}

rf = GridSearchCV(pipeline,parameter_rf,cv=3,n_jobs=-1,verbose=1)
rf.fit(X_train,y_train)

# Evaluation
print(rf.best_params_)
print(rf.score(X_train,y_train),rf.best_score_,rf.score(X_test,y_test))

## Logistic Regression

In [ ]:
preprosesor = ColumnTransformer([("numeric",num_pipe,['age','trtbps','chol','thalachh','oldpeak','thall']),
                              
                               ("category",cat_pipe,['sex','fbs','cp','restecg','exng', 'slp','caa'])])


pipeline = Pipeline([("prep",preprosesor),
                    ("algo",LogisticRegression(multi_class='multinomial',solver='lbfgs',n_jobs=-1,random_state=42))])

parameter_logreg = {'algo__fit_intercept': [True, False],
 'algo__C': np.logspace(-3,3,7)}

logreg = GridSearchCV(pipeline,parameter_logreg,cv=3,n_jobs=-1,verbose=1)
logreg.fit(X_train,y_train)

# Evaluation
print(logreg.best_params_)
print(logreg.score(X_train,y_train),logreg.best_score_,logreg.score(X_test,y_test))

# XGBoost

In [ ]:
preprosesor = ColumnTransformer([("numeric",num_pipe,['age','trtbps','chol','thalachh','oldpeak','thall']),
                              
                               ("category",cat_pipe,['sex','fbs','cp','restecg','exng', 'slp','caa'])])


pipeline = Pipeline([("prep",preprosesor),
                    ("algo",XGBClassifier(random_state=42,n_jobs=-1))])

parameter_xgb = {'algo__max_depth': Integer(low=1, high=10),
 'algo__learning_rate': Real(low=-2, high=0, prior='log-uniform'),
 'algo__n_estimators': Integer(low=100, high=200),
 'algo__subsample': Real(low=0.3, high=0.8, prior='uniform'),
 'algo__gamma': Integer(low=1, high=10),
 'algo__colsample_bytree': Real(low=0.1, high=1, prior='uniform'),
 'algo__reg_alpha': Real(low=-3, high=1, prior='log-uniform'),
 'algo__reg_lambda': Real(low=-3, high=1, prior='log-uniform')}

xgb = RandomizedSearchCV(pipeline,parameter_xgb,n_iter=150,cv=3,n_jobs=-1,verbose=1)
xgb.fit(X_train,y_train)

# Evaluation
print(xgb.best_params_)
print(xgb.score(X_train,y_train),xgb.best_score_,xgb.score(X_test,y_test))

# Evaluation Models

In [ ]:
models = [svc,knn,rf,logreg,xgb]
train_accuracy = []
test_accuracy = []

for i in models:
    y_pred_train = i.predict(X_train)
    accuracy_train = accuracy_score(y_train,y_pred_train)
    train_accuracy.append(accuracy_train)
    y_pred_test = i.predict(X_test)
    accuracy_test = accuracy_score(y_test,y_pred_test)
    test_accuracy.append(accuracy_test)

# Accuracy Train

In [ ]:
plt.figure(figsize=(10,8))
plt.bar(['SVC', 'KNN', 'RF', 'LOGREG', 'XGB'], train_accuracy)
plt.title('Accuracy Train Data comparision for various models', fontsize=15, color='r')
plt.xlabel('Models', fontsize=18, color='g')
plt.ylabel('Accuracy Score', fontsize=18, color='g')
plt.tight_layout()
plt.show()

# Accuracy Test

In [ ]:
plt.figure(figsize=(10,8))
plt.bar(['SVC', 'KNN', 'RF', 'LOGREG', 'XGB'], test_accuracy)
plt.title('Accuracy Test Data comparision for various models', fontsize=15, color='r')
plt.xlabel('Models', fontsize=18, color='g')
plt.ylabel('Accuracy Score', fontsize=18, color='g')
plt.tight_layout()
plt.show()